In [ ]:
library(dplyr)
library(ggplot2)
library(latex2exp)
library(gridExtra)

In [2]:
data <- read.csv("data/timeseries.csv")
params <- read.csv("data/params.csv")

In [3]:
nrows <- 10
nsims <- nrows^2

selected_sims <- 1:nsims

In [10]:
# Filter the dataset based on the selected simulations and integer range
filtered_data <- data %>%
    filter(sim_number %in% selected_sims, integer %in% 1:10)

max_time <- max(filtered_data$time)
filtered_data <- filtered_data %>%
    filter(time == max_time)

In [11]:
# Aggregate data: sum frequency over `chemostat_id`
summarised_data <- filtered_data %>%
  group_by(sim_number, time, integer) %>%
  summarise(frequency = sum(frequency), .groups = "drop")

In [38]:
# Create the plot
p <- ggplot(summarised_data, aes(x = factor(integer), y = frequency)) +
    geom_bar(stat = "identity", fill = "skyblue", width = 0.7) +
    scale_y_log10() +
    labs(title = "Simulation Results", x = NULL, y = NULL) +
    theme_minimal() +
    theme(legend.position = "none",
          plot.title = element_text(hjust = 0.5)) +
    facet_wrap(~ sim_number, labeller = labeller(sim_number = function(x) {
        sim_params <- params %>% filter(sim_number == x)
        sprintf("I=%.2e \n kd=%.2e", sim_params$inflow_mols, sim_params$outflow_rate)
    })) +
    geom_text(aes(label = sim_number), x = Inf, y = Inf, hjust = 1.5, vjust = 1.5, size = 10, color = "grey", alpha = 0.5)

# Save the figure
ggsave("figs/multipanel-histograms.pdf", plot = p, width = 20, height = 20)
